# Phenotype Simulator and Transfer Learning Tutorial 

The following tutorial walks through using the additive and interaction effect phenotypes simulator and then using the command line softwares to train MLP and CNN source networks and MLP and CNN transfer learning networks. 

The toy data set used in this example was simulated using HAPGEN2 with the GBR and YRI populations from the 1000 Genomes project used as the reference datasets. For the source set, 1200 GBR samples were simulated. For the target set, 300 YRI samples were simulated. This is split into the training, validation, and test for the source set resulting in 1000, 100, and 100 samples, respectively, and for the target set split into 100, 100, and 100 samples, respectively. Only genetic data for chromosome 22 was simulated. 



## Step 1: Download the toy data set and necessary softwares

* GitHub repository: https://github.com/megan-duff/transfer-learning-for-equitable-research
    - This GitHub contains the toy dataset and the softwares to simulate phenotypes with additive and interaction effects and train the source and transfer learning networks. 

* PLINK2 
    - PLINK2 is a statistical genetics software that performs common statistical analyses. We will use this to compute allele frequencies and perform a GWAS. 

In [ ]:
# Download GitHub repository and specify where to download this repository (PATH_TO_DOWNLOAD)
PATH_TO_DOWNLOAD="~/Desktop/TL-for-equitable-research_github"
git clone https://github.com/megan-duff/transfer-learning-for-equitable-research ${PATH_TO_DOWNLOAD}

# Download PLINK2 software (#https://www.cog-genomics.org/plink/2.0/)
wget https://s3.amazonaws.com/plink2-assets/plink2_linux_avx2_20240818.zip 
unzip plink2_linux_avx2_20240818.zip

## Step 2: Phenotype Simulation

To perform phenotype simulation and to train the neural networks, various files are needed. The following steps show how to create the neccessary files needed for phenotype simulation and to perform the simulation. This includes:

1. Create .afreq files for the source and target populations.
2. Create list of causal SNPs.
3. Create simulated phenotype files.
4. Split simulated phenotype files into train/validation/test specific set files. 

The following steps will show how to create each of these files and perform phenotype simulation from the toy_data provided in the GitHub repository. 

NOTE: These files and example phenotypes are provided in the toy dataset. If you wish to skip this step and start training neural network models, go ahead to Step 3: Source Neural Networks. 

#### 1. Create .afreq files for the source and target populations.
Compute the allele frequencies for the source and target populations. This is used to standardize the data. 

In [ ]:
# Compute AF for full set
toy_example_directory=${PATH_TO_DOWNLOAD}/toy_example
./plink2 --bfile ${toy_example_directory}/GBR_chr_22_1200_samples_merged --freq --out ${toy_example_directory}/GBR_chr_22_1200_samples_merged
./plink2 --bfile ${toy_example_directory}/YRI_chr_22_300_samples_merged --freq --out ${toy_example_directory}/YRI_chr_22_300_samples_merged

#Compute AF for training set
./plink2 --bfile ${toy_example_directory}/GBR_chr_22_1000_samples_train --freq --out ${toy_example_directory}/GBR_chr_22_1000_samples_train
./plink2 --bfile ${toy_example_directory}/YRI_chr_22_100_samples_train --freq --out ${toy_example_directory}/YRI_chr_22_100_samples_train

#### 2. Create list of causal SNPs.

We will select 100 random SNPs across chromsome 22 to be our causal SNPs used in phenotype simulation. The following R script provided randomly selects 100 SNPs and saves the rsID numbers to a .txt file.

In [ ]:
import pandas as pd
import numpy as np

# Load allele frequency data
toy_example_dataset="~/Desktop/TL-for-equitable-research_github"
GBR_af = pd.read_csv(f"{toy_example_dataset}/GBR_chr_22_1200_samples_merged.afreq", delim_whitespace=True, header=None, skiprows=1)
YRI_af = pd.read_csv(f"{toy_example_dataset}/YRI_chr_22_300_samples_merged.afreq", delim_whitespace=True, header=None, skiprows=1)

# Create a master DataFrame with the allele frequencies
merged_af = pd.merge(GBR_af, YRI_af, on=1, suffixes=('_GBR', '_YRI'))
merged_af_subset = merged_af[[1, '5_GBR', '5_YRI']]

# Remove rows with any zero allele frequencies
master_af_subset_remove_zero = merged_af_subset[(merged_af_subset.iloc[:, 1:3] != 0).all(axis=1)]
master_af_subset_remove_zero.columns = ['rsID', 'GBR_af', 'YRI_af']

# List of SNP counts
snp_list = [100]

# Sample SNPs and write them to files
for num_of_snps in snp_list:
    sample = master_af_subset_remove_zero['rsID'].sample(num_of_snps)
    sample.to_csv(f"{toy_example_dataset}/{num_of_snps}_causal_snps.txt", index=False, header=False)

print("Finished causal SNP selection!")


#### 3. Create simulated phenotype files.

Next, we simulate the additive and interaction effects phenotype for the source and target sets. 

We will simulate phenotypes with 100 causal SNPs in total, an overall heritability of 0.5 with 5% of that heritability attributed to interaction effects (resulting in phenotypes with an additive heritability of 0.475 and an interaction heritability of 0.025), with 100 interaction terms. These interaction terms are randomly selected from the set of causal SNPs, where pairs of SNPs are randomly selected and their product is used as the interaction term. 

The first code simulates the phenotype for the source set while the second code chunk simulates the phenotype for the target set. 

Inputs:

* --plink_file: Prefix for the PLINK data files (.bed/.bim/.fam)
* --af_file: Allele frequency file computed by PLINK2 (.afreq format)
* --causal_snp_file: Causal SNP file (one column with rsIDs that correspond to causal SNPs)
* --h2: Overall heritability of the phenotype
* --e 0.05: Proportion of heritability attributed to interaction effects 
* --num_interaction_terms: Number of pairs of SNPs to randomly select (their product is the interaction term in the model)
* --output: Prefix to use for files

In [ ]:
pheno_sim_directory=${PATH_TO_DOWNLOAD}/Softwares/phenotype_simulators

Rscript ${pheno_sim_directory}/source_additive_and_interaction_effects_phenotype_simulator.R \
--plink_file ${toy_example_directory}/GBR_chr_22_1200_samples_merged \
--af_file ${toy_example_directory}/GBR_chr_22_1200_samples_merged.afreq \
--causal_snp_file ${toy_example_directory}/100_causal_snps.txt \
--h2 0.5 \
--e 0.05 \
--num_interaction_terms 100 \
--output ${toy_example_directory}/GBR_chr_22_1200_samples_merged_non_linear_phenotypes

In [ ]:
Rscript ${pheno_sim_directory}/target_additive_and_interaction_effects_phenotype_simulator.R \
--plink_file ${toy_example_directory}/YRI_chr_22_300_samples_merged \
--af_file ${toy_example_directory}/YRI_chr_22_300_samples_merged.afreq \
--source_causal_snp_file ${toy_example_directory}/100_causal_snps.txt \
--h2 0.5 \
--e 0.05 \
--num_interaction_terms 100 \
--output ${toy_example_directory}/YRI_chr_22_300_samples_merged_non_linear_phenotypes \
--source_par_file ${toy_example_directory}/GBR_chr_22_1200_samples_merged_non_linear_phenotypes.par \
--source_interaction_file ${toy_example_directory}/GBR_chr_22_1200_samples_merged_non_linear_phenotypes_interactions.txt \
--genetic_correlation=0.8 

#### 4. Split simulated phenotype files into train/validation/test specific set files.

Create phenotype files for training, validation, and test set. 

The data provided 

In [ ]:
cut -d' ' -f1 ${toy_example_directory}/GBR_chr_22_1000_samples_train.fam > ${toy_example_directory}/GBR_chr_22_1000_samples_train_samples.txt
cut -d' ' -f1 ${toy_example_directory}/GBR_chr_22_100_samples_val.fam > ${toy_example_directory}/GBR_chr_22_100_samples_val_samples.txt
cut -d' ' -f1 ${toy_example_directory}/GBR_chr_22_100_samples_test.fam > ${toy_example_directory}/GBR_chr_22_100_samples_test_samples.txt

cut -d' ' -f1 ${toy_example_directory}/YRI_chr_22_100_samples_train.fam > ${toy_example_directory}/YRI_chr_22_100_samples_train_samples.txt
cut -d' ' -f1 ${toy_example_directory}/YRI_chr_22_100_samples_val.fam > ${toy_example_directory}/YRI_chr_22_100_samples_val_samples.txt
cut -d' ' -f1 ${toy_example_directory}/YRI_chr_22_100_samples_test.fam > ${toy_example_directory}/YRI_chr_22_100_samples_test_samples.txt

In [ ]:
#!/usr/bin/env Rscript

toy_example_dataset="~/Desktop/TL-for-equitable-research_github"

train_samples<-read.table(f'{toy_example_dataset}/GBR_chr_22_1000_samples_train_samples.txt', header=FALSE)
val_samples<-read.table(f'{toy_example_dataset}/GBR_chr_22_100_samples_val_samples.txt', header=FALSE)
test_samples<-read.table(f'{toy_example_dataset}/GBR_chr_22_100_samples_test_samples.txt', header=FALSE)

pheno= read.table(f"{toy_example_dataset}/GBR_chr_22_1200_samples_merged_non_linear_phenotypes.phen", header = T, sep="\t")

colnames(pheno) = c("ID", "ID2", "Phenotype")
    
pheno_train <- pheno[pheno$ID %in% train_samples$V1,]
pheno_val <- pheno[pheno$ID %in% val_samples$V1,]
pheno_test <- pheno[pheno$ID %in% test_samples$V1,]
    
pheno_train_file = f"{toy_example_dataset}/GBR_chr_22_1000_samples_train_non_linear_phenotypes.phen"
pheno_val_file = f"{toy_example_dataset}/GBR_chr_22_100_samples_val_non_linear_phenotypes.phen"
pheno_test_file = f"{toy_example_dataset}/GBR_chr_22_100_samples_test_non_linear_phenotypes.phen"
    
write.table(pheno_train, file = pheno_train_file, row.names = FALSE, col.names = FALSE, quote=FALSE, sep='\t')
write.table(pheno_val, file = pheno_val_file, row.names = FALSE, col.names = FALSE, quote=FALSE, sep='\t')
write.table(pheno_test, file = pheno_test_file, row.names = FALSE, col.names = FALSE, quote=FALSE, sep='\t')
    

In [ ]:
#!/usr/bin/env Rscript

toy_example_dataset="~/Desktop/TL-for-equitable-research_github"

train_samples<-read.table(f'{toy_example_dataset}/YRI_chr_22_100_samples_train_samples.txt', header=FALSE)
val_samples<-read.table(f'{toy_example_dataset}/YRI_chr_22_100_samples_val_samples.txt', header=FALSE)
test_samples<-read.table(f'{toy_example_dataset}/YRI_chr_22_100_samples_test_samples.txt', header=FALSE)

pheno= read.table(f"{toy_example_dataset}/YRI_chr_22_300_samples_merged_non_linear_phenotypes.phen", header = F, sep="\t")

colnames(pheno) = c("ID", "ID2", "Phenotype")
    
pheno_train <- pheno[pheno$ID %in% train_samples$V1,]
pheno_val <- pheno[pheno$ID %in% val_samples$V1,]
pheno_test <- pheno[pheno$ID %in% test_samples$V1,]
    
pheno_train_file = f"{toy_example_dataset}/YRI_chr_22_100_samples_train_non_linear_phenotypes.phen"
pheno_val_file = f"{toy_example_dataset}/YRI_chr_22_100_samples_val_non_linear_phenotypes.phen"
pheno_test_file = f"{toy_example_dataset}/YRI_chr_22_100_samples_test_non_linear_phenotypes.phen"
    
write.table(pheno_train, file = pheno_train_file, row.names = FALSE, col.names = FALSE, quote=FALSE, sep='\t')
write.table(pheno_val, file = pheno_val_file, row.names = FALSE, col.names = FALSE, quote=FALSE, sep='\t')
write.table(pheno_test, file = pheno_test_file, row.names = FALSE, col.names = FALSE, quote=FALSE, sep='\t')

## Step 3: Source MLP and CNN Tutorial 

Now that we have genetic and phenotypic data, the next step is to build the source networks. Two networks will be built, one of the MLP architecture and the CNN architecture. 

Prior to model building, using the simulated phenotypes, and the training datasets, we will perform a GWAS to use in pre-selecting SNPs for the neural network models. 

In [ ]:
# Compute PCs to use in GWAS
./plink2 --bfile ${toy_example_directory}/GBR_chr_22_1000_samples_train --pca --out ${toy_example_directory}/GBR_chr_22_1000_samples_train
# Subset to top 10 PCs
cut -d' ' -f1-12 ${toy_example_directory}/GBR_chr_22_1000_samples_train.eigenvec > ${toy_example_directory}/covariates.txt

# Add header to phenotype file to work in PLINK2
# Step 1: Create a temporary file with the header
echo -e "FID\tIID\tPHENO2" > ${toy_example_directory}/temp_header.txt
# Step 2: Concatenate the header with the original file
cat ${toy_example_directory}/temp_header.txt ${toy_example_directory}/GBR_chr_22_1200_samples_merged_non_linear_phenotypes.phen > ${toy_example_directory}/temp_combined.phen
# Step 3: Replace the original file with the new one
mv ${toy_example_directory}/temp_combined.phen ${toy_example_directory}/GBR_chr_22_1200_samples_merged_non_linear_phenotypes.phen
# Step 4: Remove the temporary header file
rm ${toy_example_directory}/temp_header.txt

# Run GWAS
./plink2 --bfile ${toy_example_directory}/GBR_chr_22_1000_samples_train --pheno ${toy_example_directory}/GBR_chr_22_1200_samples_merged_non_linear_phenotypes.phen --pheno-name PHENO2 --glm --covar ${toy_example_directory}/covariates.txt --covar-col-nums 3-12 --out ${toy_example_directory}/GBR_chr_22_1000_samples_train_non_linear_phenotypes

To build the source MLP and CNN models, the softwares are provided in the github repository in the Softwares/genomic-mlp/ and Softwares/genomic-cnn/ directory. 

First the appropriate software is downloaded and dependencies are installed. 

In [ ]:
cd ${PATH_TO_DOWNLOAD}/Softwares/genomic-mlp/
pip install .

cd ${PATH_TO_DOWNLOAD}/Softwares/genomic-cnn/
pip install .

Now, we can use the software. 

The following provides the inputs and outputs for both the source MLP and CNN softwares. 

##### Inputs:
--train_prefix, --val_prefix, --test_prefix : Path to PLINK bed/bim/fam file prefixes.

Expects PLINK1.9 format for each of these files
* .bed: https://www.cog-genomics.org/plink/1.9/formats#bed
* .bim: https://www.cog-genomics.org/plink/1.9/formats#bim
* .fam: https://www.cog-genomics.org/plink/1.9/formats#fam

--train_pheno_file, --val_pheno_file, --test_pheno_file: Path to phenotype files.

Expects phenotype files are in the following format:
* No header 
* 3 columns 
    * 1st column: FID
    * 2nd column: IID
    * 3rd column: Phenotype Value
* Ensure the FID and IID matches the individuals in the correpsonding PLINK files

--snp_file: GWAS summary statistics file 
* Created from PLINK2 --glm analysis (https://www.cog-genomics.org/plink/2.0/formats#glm_linear).

--train_af_file: Allele frequency file 
* Created from PLINK2 --freq analysis (https://www.cog-genomics.org/plink/2.0/formats#afreq).

--epochs: Number of epochs used for hyperparameter search and to train neural network.

--batch_size: Batch size value used for hyperparameter search and to train neural network.

--output_dir: Directory to save results, if not specified the current directory is used. 

--seed: Set the seed, if seed is not set then a random seed is used.

##### Outputs:

* MLP_best_hp_model.obj / CNN_best_hp_model.obj: File that saves the best hyperparameters found during the hyperparameter optimization process.


* MLP_best_hp_model_params.obj / CNN_best_hp_model_params.obj: The file will contain the best model created in hyperparameter training in a binary format. This saved model can be loaded later to make predictions, continue training, or perform further evaluation.


* MLP_loss_vs_epochs.png / CNN_loss_vs_epochs.png: The loss vs. epoch plot. 


* MLP_results.csv / CNN_results.csv: Results CSV containing various metrics including the training, validation, and test correlation. 


* MLP_trained_best_model.keras / CNN_trained_best_model.keras: A .keras model file is a file format used by TensorFlow's Keras API to save trained neural network models. This file stores the entire model architecture, including the layers, weights, optimizer, and configuration, allowing the model to be easily reloaded and used for inference or further training.


* nn_training/: This directory contains all the metadata created during hyperparameter optimization. 

In [ ]:
${PATH_TO_DOWNLOAD}/Softwares/genomic-mlp/genomic-mlp \
--train_prefix ${toy_example_directory}/GBR_chr_22_1000_samples_train \
--val_prefix ${toy_example_directory}/GBR_chr_22_100_samples_val \
--test_prefix ${toy_example_directory}/GBR_chr_22_100_samples_test \
--snp_file ${toy_example_directory}/GBR_chr_22_1000_samples_train_non_linear_phenotypes.PHENO2.glm.linear \
--train_af_file ${toy_example_directory}/GBR_chr_22_1000_samples_train.afreq \
--train_pheno_file ${toy_example_directory}/GBR_chr_22_1000_samples_train_non_linear_phenotypes.phen \
--val_pheno_file ${toy_example_directory}/GBR_chr_22_100_samples_val_non_linear_phenotypes.phen \
--test_pheno_file ${toy_example_directory}/GBR_chr_22_100_samples_test_non_linear_phenotypes.phen \
--epochs 100 \
--batch-size 64 \
--output_dir ${toy_example_directory}/source_mlp/ \
--seed 5

In [ ]:
${PATH_TO_DOWNLOAD}/Softwares/genomic-cnn/genomic-cnn \
--train_prefix ${toy_example_directory}/GBR_chr_22_1000_samples_train \
--val_prefix ${toy_example_directory}/GBR_chr_22_100_samples_val \
--test_prefix ${toy_example_directory}/GBR_chr_22_100_samples_test \
--snp_file ${toy_example_directory}/GBR_chr_22_1000_samples_train_non_linear_phenotypes.PHENO2.glm.linear \
--train_af_file ${toy_example_directory}/GBR_chr_22_1000_samples_train.afreq \
--train_pheno_file ${toy_example_directory}/GBR_chr_22_1000_samples_train_non_linear_phenotypes.phen \
--val_pheno_file ${toy_example_directory}/GBR_chr_22_100_samples_val_non_linear_phenotypes.phen \
--test_pheno_file ${toy_example_directory}/GBR_chr_22_100_samples_test_non_linear_phenotypes.phen \
--epochs 100 \
--batch-size 64 \
--output_dir ${toy_example_directory}/source_cnn/ \
--seed 5

## Step 4: Transfer Learning MLP and CNN Tutorial 

To build the transfer learning MLP and CNN models, the softwares are provided in the github repository in the Softwares/genomic-mlp-transfer-learning/ and Softwares/genomic-cnn-transfer-learning/ directory. 

First the appropriate software is downloaded and dependencies are installed. 

In [ ]:
cd ${PATH_TO_DOWNLOAD}/Softwares/genomic-mlp-transfer-learning/
pip install .

cd ${PATH_TO_DOWNLOAD}/Softwares/genomic-cnn-transfer-learning/
pip install .

Now, we can use the software. 

The following provides the inputs and outputs for both the transfer learning MLP and CNN softwares. 

##### Inputs:
--train_prefix, --val_prefix, --test_prefix : Path to PLINK bed/bim/fam file prefixes.

Expects PLINK1.9 format for each of these files
* .bed: https://www.cog-genomics.org/plink/1.9/formats#bed
* .bim: https://www.cog-genomics.org/plink/1.9/formats#bim
* .fam: https://www.cog-genomics.org/plink/1.9/formats#fam

--train_pheno_file, --val_pheno_file, --test_pheno_file: Path to phenotype files.

Expects phenotype files are in the following format:
* No header 
* 3 columns 
    * 1st column: FID
    * 2nd column: IID
    * 3rd column: Phenotype Value
* Ensure the FID and IID matches the individuals in the correpsonding PLINK files

--source_snp_file: GWAS summary statistics file 
* Created from PLINK2 --glm analysis (https://www.cog-genomics.org/plink/2.0/formats#glm_linear).

--train_af_file: Allele frequency file 
* Created from PLINK2 --freq analysis (https://www.cog-genomics.org/plink/2.0/formats#afreq).

--source_model_best_params_file: File for the best parameter found in the hyperparameter search of the source set.

--source_model_keras_file: File for the best trained model (.keras)

--epochs: Number of epochs used for hyperparameter search and to train neural network.

--batch_size: Batch size value used for hyperparameter search and to train neural network.

--output_dir: Directory to save results, if not specified the current directory is used. 

--seed: Set the seed, if seed is not set then a random seed is used.

##### Outputs:

* MLP_TL_fine_tune_loss_vs_epochs_plot.png / CNN_TL_fine_tune_loss_vs_epochs_plot.png: The loss vs. epoch plot. 


* MLP_results.csv / CNN_results.csv: Results CSV containing various metrics including the training, validation, and test correlation. 


* MLP_trained_best_model.keras / CNN_trained_best_model.keras: A .keras model file is a file format used by TensorFlow's Keras API to save trained neural network models. This file stores the entire model architecture, including the layers, weights, optimizer, and configuration, allowing the model to be easily reloaded and used for inference or further training.


* nn_training/: This directory contains all the metadata created during hyperparameter optimization. 

In [ ]:
${PATH_TO_DOWNLOAD}/Softwares/genomic-mlp-transfer-learning/genomic-mlp-transfer-learning \
--train_prefix ${toy_example_directory}/YRI_chr_22_100_samples_train \
--val_prefix ${toy_example_directory}/YRI_chr_22_100_samples_val \
--test_prefix ${toy_example_directory}/YRI_chr_22_100_samples_test \
--source_snp_file ${toy_example_directory}/GBR_chr_22_1000_samples_train_non_linear_phenotypes.PHENO2.glm.linear \
--train_af_file ${toy_example_directory}/YRI_chr_22_100_samples_train.afreq \
--train_pheno_file ${toy_example_directory}/YRI_chr_22_100_samples_train_non_linear_phenotypes.phen \
--val_pheno_file ${toy_example_directory}/YRI_chr_22_100_samples_val_non_linear_phenotypes.phen \
--test_pheno_file ${toy_example_directory}/YRI_chr_22_100_samples_test_non_linear_phenotypes.phen \
--source_model_best_params_file ${toy_example_directory}/source_mlp/MLP_best_hp_model_params.obj  \
--source_model_keras_file ${toy_example_directory}/source_mlp/MLP_trained_best_model.keras \
--epochs 100 \
--batch_size 64 \
--output_dir ${toy_example_directory}/transfer_learning_MLP/ \
--seed 5

In [ ]:
${PATH_TO_DOWNLOAD}/Softwares/genomic-cnn-transfer-learning/genomic-cnn-transfer-learning \
--train_prefix ${toy_example_directory}/YRI_chr_22_100_samples_train \
--val_prefix ${toy_example_directory}/YRI_chr_22_100_samples_val \
--test_prefix ${toy_example_directory}/YRI_chr_22_100_samples_test \
--source_snp_file ${toy_example_directory}/GBR_chr_22_1000_samples_train_non_linear_phenotypes.PHENO2.glm.linear \
--train_af_file ${toy_example_directory}/YRI_chr_22_100_samples_train.afreq \
--train_pheno_file ${toy_example_directory}/YRI_chr_22_100_samples_train_non_linear_phenotypes.phen \
--val_pheno_file ${toy_example_directory}/YRI_chr_22_100_samples_val_non_linear_phenotypes.phen \
--test_pheno_file ${toy_example_directory}/YRI_chr_22_100_samples_test_non_linear_phenotypes.phen \
--source_model_best_params_file ${toy_example_directory}/source_cnn/CNN_best_hp_model_params.obj  \
--source_model_keras_file ${toy_example_directory}/source_cnn/CNN_trained_best_model.keras \
--epochs 100 \
--batch_size 64 \
--output_dir ${toy_example_directory}/transfer_learning_CNN/ \
--seed 5